In [1]:
import os
import rioxarray as rxr
from shapely.geometry import mapping
import geopandas as gpd
import rasterio as rio
from rioxarray.merge import merge_arrays

C:\Users\enhi\AppData\Local\Temp\ipykernel_16728\4077202506.py:4: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopandas as gpd


In [2]:
main_dir = r'C:\Users\enhi\OneDrive - gmv.com\Documents\Projects\ANIN\Generating Indices\VCI\NDVI-16\Trial\Tiles'
#Tiles_path = os.path.join(main_dir, 'Tiles')
Tile_1 = os.path.join(main_dir, 'tile_1')
Tile_2 = os.path.join(main_dir, 'tile_2')
Tile_3 = os.path.join(main_dir, 'tile_3')
Tile_4 = os.path.join(main_dir, 'tile_4')

Images_in_tile1 = sorted(os.listdir(Tile_1))
Images_in_tile2 = sorted(os.listdir(Tile_2))
Images_in_tile3 = sorted(os.listdir(Tile_3))
Images_in_tile4 = sorted(os.listdir(Tile_4))

In [3]:
Boundary = os.path.join(main_dir, 'Boundary')
South_Africa = gpd.read_file(Boundary + '\\'+ 'ZAF_adm0.shp')
lesotho = gpd.read_file(Boundary + '\\'+ 'LSO_adm0.shp')
eswatini = gpd.read_file(Boundary +'\\'+  'swz_admbnda_adm0_CSO_2007.shp')
merged = South_Africa.geometry.append(eswatini.geometry)
merged =merged.geometry.append(lesotho.geometry)

C:\Users\enhi\Anaconda3\envs\rasterio\Lib\site-packages\geopandas\geoseries.py:622: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  val = getattr(super(), mtd)(*args, **kwargs)


In [5]:
Clipped_path = os.path.join(main_dir, 'Clipped_trial')
for i in range (0 , len(Images_in_tile4)):
    tile1 = rxr.open_rasterio(Tile_1 + '\\'+ Images_in_tile1[i], masked=True)
    tile2 = rxr.open_rasterio(Tile_2 + '\\'+ Images_in_tile2[i], masked=True)
    tile3 = rxr.open_rasterio(Tile_3 + '\\'+ Images_in_tile3[i], masked=True)
    tile4 = rxr.open_rasterio(Tile_4 + '\\'+ Images_in_tile4[i], masked=True)
    merged_raster = merge_arrays(dataarrays = [tile1['250m 16 days NDVI'], tile2['250m 16 days NDVI'],
                                               tile3['250m 16 days NDVI'],tile4['250m 16 days NDVI']])
    clipped = merged_raster.rio.clip(merged.geometry.apply(mapping),
                                     crs=merged.crs,
                                     all_touched=True,
                                     from_disk=True).squeeze()
    year = Clipped_path + '\\'+ Images_in_tile1[i][9:13]
    check_folder_year = os.path.isdir(year)
    if not check_folder_year:
        os.makedirs(year)
    
    clipped.rio.to_raster(year + '\\'+ Images_in_tile1[i]+ '.tif')